In [ ]:
import subprocess
import time
import csv
import re
def parse_nvidia_smi_output(output):
    data = {
        'timestamp': '',
        'gpu_utilization': '',
        'memory_utilization': '',
        'encoder_utilization': '',
        'decoder_utilization': ''
    }

    # Regular expressions for extracting data
    timestamp_pattern = re.compile(r"Timestamp\s+:\s+(.*)")
    gpu_pattern = re.compile(r"Gpu\s+:\s+(\d+)\s*%")
    memory_pattern = re.compile(r"Memory\s+:\s+(\d+)\s*%")
    encoder_pattern = re.compile(r"Encoder\s+:\s+(\d+)\s*%")
    decoder_pattern = re.compile(r"Decoder\s+:\s+(\d+)\s*%")

    # Extract the timestamp
    timestamp_match = timestamp_pattern.search(output)
    if timestamp_match:
        data['timestamp'] = timestamp_match.group(1)

    # Extract utilization data
    gpu_match = gpu_pattern.search(output)
    if gpu_match:
        data['gpu_utilization'] = gpu_match.group(1)

    memory_match = memory_pattern.search(output)
    if memory_match:
        data['memory_utilization'] = memory_match.group(1)

    encoder_match = encoder_pattern.search(output)
    if encoder_match:
        data['encoder_utilization'] = encoder_match.group(1)

    decoder_match = decoder_pattern.search(output)
    if decoder_match:
        data['decoder_utilization'] = decoder_match.group(1)

    return data

# Initialize CSV file and write header if it does not exist
def initialize_csv(csv_file):
    try:
        with open(csv_file, 'x', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['timestamp', 'gpu_utilization', 'memory_utilization', 'encoder_utilization', 'decoder_utilization'])
            writer.writeheader()
    except FileExistsError:
        pass

# Append data to the CSV file
def append_to_csv(data,csv_file):
    with open(csv_file, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['timestamp', 'gpu_utilization', 'memory_utilization', 'encoder_utilization', 'decoder_utilization'])
        writer.writerow(data)

In [ ]:
import abc
import time
import threading
import subprocess
import pandas as pd
import os

class EvaluationMetrics(metaclass=abc.ABCMeta):
    def __init__(self,predictor:object,images_path:str,labels_path:str) -> None:
        self.__predictor=predictor #model object
        self.__images_path=images_path #path to images dir
        self.__labels_path=labels_path #path to labels dir
        self._monitoring = False #flag to wether gpu is being monitored
    
    #predict using self.__predictor the self.__labels_path of the self.__images_path
    @abc.abstractmethod
    def predict(self,one=False, image_path="")->list:
        pass
        #implement according to you model

    #write gpu utilization to csv
    @staticmethod
    def monitor_gpu(csv_file)->None:
        initialize_csv(csv_file)
        while True:
            # Run the nvidia-smi command
            result = subprocess.run(['nvidia-smi', '-q', '-i', '0', '-d', 'UTILIZATION'], capture_output=True, text=True)
            output = result.stdout

            # Parse the output
            data = parse_nvidia_smi_output(output)

            # Append the data to CSV
            append_to_csv(data,csv_file)

            # Wait before the next read
            time.sleep(1)

    #evaluate gpu utilization
    def gpu_evaluation(self,csv_file_path:str )->None:
        self._monitoring = True
        monitor_thread = threading.Thread(target=self.monitor_gpu, args=(csv_file_path,))
        monitor_thread.start()

        self.predict()

        self._monitoring = False
        monitor_thread.join()  # Wait for the monitoring thread to finish

    #evaluate time for predictions
    def time_evaluation(self)->float:
        start=time.time()
        self.predict()
        end=time.time()
        return end-start
    


    #needs to be decided which metrics to implement
    
    #precision
    def metric_1(self,csv_file)->float:

        total_true_positive = 0
        total_ground_truth = 0

        ground_truth=self.load_ground_truth(csv_file)
        for image_name, gt_boxes in list(ground_truth.items())[:100]:
            # image_path = os.path.join(self.__images_path)
            predictions = self.predict(one=True, image_path=image_name)[0]
            
            pred_boxes = predictions.pred_boxes.tensor.numpy()
            
            # Check if any predictions overlap with ground truth
            for gt_box in gt_boxes:
                gt_xmin, gt_ymin, gt_xmax, gt_ymax = gt_box
                match_found = False
                for pred_box in pred_boxes:
                    if self.check_overlap((gt_xmin, gt_ymin, gt_xmax, gt_ymax), tuple(pred_box)):
                        match_found = True
                        break
                if match_found:
                    total_true_positive += 1
                total_ground_truth += 1
        
        accuracy = total_true_positive / total_ground_truth if total_ground_truth > 0 else 0
    
        return accuracy
    
    def check_overlap(self,box1, box2):
        x1, y1, x2, y2 = box1
        x1_p, y1_p, x2_p, y2_p = box2

        # Check if there is any overlap
        if x1 < x2_p and x2 > x1_p and y1 < y2_p and y2 > y1_p:
            return True
        return False
    
    #true classes
    def load_ground_truth(self,csv_file):
        df = pd.read_csv(csv_file)
        ground_truth = {}
        for _, row in df.iterrows():
            # Extract the necessary information from the row
            image_name = row["image_path"]
            xmin = int(row["xmin"])
            ymin = int(row["ymin"])
            xmax = int(row["xmax"])
            ymax = int(row["ymax"])

            # Add the bounding box information to the ground_truth dictionary
            if image_name not in ground_truth:
                ground_truth[image_name] = []
            ground_truth[image_name].append((xmin, ymin, xmax, ymax))
        return ground_truth 

    def metric_2(self):
        pass

    def metric_3(self):
        pass

